# Initialization

## Importing libs and setting plot style

In [1]:
import multiprocessing as mp
import os
import threading
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from matplotlib import rc

In [9]:
# Setting plot style

sns.set()
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.0})

rc("text", usetex=True)

sns.set_style("ticks")
sns.set_style(
    "whitegrid",
    {
        "axes.edgecolor": "black",
        "axes.grid": True,
        "axes.axisbelow": True,
        "axes.labelcolor": ".15",
        "grid.color": "0.9",
        "grid.linestyle": "-",
        "xtick.direction": "in",
        "ytick.direction": "in",
        "xtick.bottom": True,
        "xtick.top": True,
        "ytick.left": True,
        "ytick.right": True,
        "font.family": ["sans-serif"],
        "font.sans-serif": ["Liberation Sans", "Bitstream Vera Sans", "sans-serif"],
    },
)

## Global variables

In [8]:
data_folder = "../data_folder"

# MH parameters
spin = 0.5
length = 1000000
sigma = 0.40
burnin = 10
verbosity = 0

# set optimal number of threads
number_of_threads = int(mp.cpu_count())
print(f"optimal number of threads: {number_of_threads}")

optimal number of threads: 12


# Angles 

In [10]:
# takes an intertwiner and returns the corresponding angle eigenvalue
def from_intertwiner_to_angle(matrix_element, spin):
    return (matrix_element * (matrix_element + 1) - 2 * spin * (spin + 1)) / (
        2 * spin * (spin + 1)
    )

In [11]:
def from_draws_to_angles(folder_prefix, spin, length, burnin, angle_path, chain_id):

    # load in memory the stored draws
    draw_path = f"{folder_prefix}/draws/draws_chain_{chain_id}.csv"
    df = pd.read_csv(draw_path, low_memory=False)

    # retrieving relevant parameters
    multeplicity = df[["draw multeplicity"]].to_numpy().astype(int)
    total_accept_draws = int(df["total accept. draws"][0])
    total_accept_rate = float(df["total accept. rate"][0].strip("%"))
    total_run_time = float(df["total run time"][0].strip(" s"))

    # dropping columns
    df = df.drop(
        columns=[
            "draw multeplicity",
            "draw amplitude",
            "total accept. draws",
            "total accept. rate",
            "total run time",
        ]
    )

    # from csv to matrix in order to use the numpy optimized routines
    angles_matrix = np.matrix(df.values.transpose()).astype(float)

    # from intertwiners to angles
    angles_matrix = np.vectorize(from_intertwiner_to_angle)(angles_matrix, spin)
    angles_sq_matrix = np.power(angles_matrix, 2)

    # from intertwiners to angles sq
    angles_matrix = np.matmul(angles_matrix, multeplicity)
    angles_sq_matrix = np.matmul(angles_sq_matrix, multeplicity)

    # average of angles
    angles_matrix = angles_matrix.sum(axis=1) / (length - burnin)
    angles_sq_matrix = angles_sq_matrix.sum(axis=1) / (length - burnin)

    nodes_angles_avg_matrix = angles_matrix.sum(axis=0) / 16
    nodes_angles_sq_avg_matrix = angles_sq_matrix.sum(axis=0) / 16

    df = pd.DataFrame(
        {
            "node 1": [angles_matrix[0, 0], angles_sq_matrix[0, 0]],
            "node 2": [angles_matrix[1, 0], angles_sq_matrix[1, 0]],
            "node 3": [angles_matrix[2, 0], angles_sq_matrix[2, 0]],
            "node 4": [angles_matrix[3, 0], angles_sq_matrix[3, 0]],
            "node 5": [angles_matrix[4, 0], angles_sq_matrix[4, 0]],
            "node 6": [angles_matrix[5, 0], angles_sq_matrix[5, 0]],
            "node 7": [angles_matrix[6, 0], angles_sq_matrix[6, 0]],
            "node 8": [angles_matrix[7, 0], angles_sq_matrix[7, 0]],
            "node 9": [angles_matrix[8, 0], angles_sq_matrix[8, 0]],
            "node 10": [angles_matrix[9, 0], angles_sq_matrix[9, 0]],
            "node 11": [angles_matrix[10, 0], angles_sq_matrix[10, 0]],
            "node 12": [angles_matrix[11, 0], angles_sq_matrix[11, 0]],
            "node 13": [angles_matrix[12, 0], angles_sq_matrix[12, 0]],
            "node 14": [angles_matrix[13, 0], angles_sq_matrix[13, 0]],
            "node 15": [angles_matrix[14, 0], angles_sq_matrix[14, 0]],
            "node 16": [angles_matrix[15, 0], angles_sq_matrix[15, 0]],
            "nodes avg": [
                nodes_angles_avg_matrix[0, 0],
                nodes_angles_sq_avg_matrix[0, 0],
            ],
        },
        index=[f"angle average", f"angle sq average"],
    )

    angle_path_chain = f"{angle_path}/angles_chain_{chain_id}.csv"
    df.to_csv(angle_path_chain, index=True)

In [12]:
def angles_compute(data_folder, spin, length, sigma, burnin, number_of_threads):

    folder_prefix = f"{data_folder}/j_{spin}/N_{length}__sigma_{sigma}__burnin_{burnin}"
    chain_id_collection = []

    for chain_id in range(1, number_of_threads + 1):
        draw_path = f"{folder_prefix}/draws/draws_chain_{chain_id}.csv"
        if os.path.isfile(draw_path):
            chain_id_collection.append(chain_id)
        else:
            warnings.warn("Warning: the draw %s was not found" % (draw_path))

    chains_to_assemble = len(chain_id_collection)

    if chains_to_assemble != 0:

        angle_path = f"{folder_prefix}/operators/angles"
        os.makedirs(angle_path, exist_ok=True)

        print(
            f"Converting {chains_to_assemble} chains from draws to averaged angles..."
        )

        threads = []
        for chain_id in chain_id_collection:
            t = threading.Thread(
                target=from_draws_to_angles,
                args=(
                    folder_prefix,
                    spin,
                    length,
                    burnin,
                    angle_path,
                    chain_id,
                ),
            )
            threads.append(t)
            t.start()

        # wait for the threads to complete
        for t in threads:
            t.join()

        print(f"Completed! All draws have been processed")

        print(f"Assembling {chains_to_assemble} chains...")

        matrix = np.zeros((chains_to_assemble, 3))

        for chain_id in range(chains_to_assemble):

            angle_path_chain = f"{angle_path}/angles_chain_{chain_id+1}.csv"
            df = pd.read_csv(angle_path_chain, index_col=0, low_memory=False)
            matrix[chain_id, 0] = df["nodes avg"]["angle average"]
            matrix[chain_id, 1] = df["nodes avg"]["angle sq average"]
            matrix[chain_id, 2] = np.sqrt(
                matrix[chain_id, 1] - matrix[chain_id, 0] * matrix[chain_id, 0]
            )

        values = np.mean(matrix, axis=0)

        angles_fluc = np.std(matrix, axis=0)[0]
        angles_computed = values[0]
        angles_sq_computed = values[1]
        angles_quantum_spread_computed = values[2]

        df = pd.DataFrame(
            {
                "angles_average": [angles_computed],
                "angles_std": [angles_fluc],
                "angles_quantum_spread": [angles_quantum_spread_computed],
                "chains_length": [length],
                "chains_assembled": [chains_to_assemble],
                "sigma": [sigma],
                "burnin": [burnin],
            },
            index=[f"j={spin}"],
        )

        assembled_angle_path = f"{data_folder}/final_tables"
        os.makedirs(assembled_angle_path, exist_ok=True)

        assembled_angle_path = f"{assembled_angle_path}/angles_j={spin}_chains_combined_{chains_to_assemble}.csv"
        df.to_csv(assembled_angle_path, index=True)

        print(f"Done")

    else:
        warnings.warn("I can't compute angles since there are no chains available")

In [13]:
angles_compute(data_folder, spin, length, sigma, burnin, number_of_threads)

Converting 12 chains from draws to averaged angles...
Completed! All draws have been processed
Assembling 12 chains...
Done


In [90]:
def angle_autocorrelation_compute(
    data_folder, node, lag_max, number_of_chains, spin, length, sigma, burnin
):
    # TODO: consider parallelize the code

    folder_prefix = f"{data_folder}/j_{spin}/N_{length}__sigma_{sigma}__burnin_{burnin}"
    chain_id_collection = []

    for chain_id in range(1, number_of_chains + 1):
        draw_path = f"{folder_prefix}/draws/draws_chain_{chain_id}.csv"
        if os.path.isfile(draw_path):
            chain_id_collection.append(chain_id)
        else:
            warnings.warn("Warning: the draw %s was not found" % (draw_path))

    chains_to_assemble = len(chain_id_collection)

    if chains_to_assemble != 0:

        angle_path = f"{folder_prefix}/operators/angles"
        os.makedirs(angle_path, exist_ok=True)

        Total_autocorrelations = np.zeros([chains_to_assemble, lag_max])
        columns = [f"chain {i}" for i in range(1, chains_to_assemble + 1)]

        print(
            f"Computing autocorrelation of node {node} over {chains_to_assemble} chains with lag max {lag_max}...\n"
        )

        folder_prefix = (
            f"{data_folder}/j_{spin}/N_{length}__sigma_{sigma}__burnin_{burnin}"
        )

        lags = [i for i in range(0, lag_max)]

        for chain_id in chain_id_collection:

            # load in memory the stored draws
            draw_path = f"{folder_prefix}/draws/draws_chain_{chain_id}.csv"
            df = pd.read_csv(draw_path, low_memory=False)

            # retrieving relevant parameters
            multeplicity = np.squeeze(df[["draw multeplicity"]].to_numpy().astype(int))
            M = multeplicity.shape[0]
            Length = sum(multeplicity)

            # from csv to array
            intertw_array_node_n = df[[f"intertwiner {node}"]].to_numpy()

            # from intertwiner to angle
            angle_array_node_n = np.squeeze(
                np.vectorize(from_intertwiner_to_angle)(intertw_array_node_n, spin)
            )

            # extended angle list
            angle_array_node_n_extended = np.zeros(Length)

            counter = 0

            for m in range(M):

                angle_array_node_n_extended[counter : (counter + multeplicity[m])] = [
                    angle_array_node_n[m] for x in range(multeplicity[m])
                ]
                counter += multeplicity[m]

            Total_autocorrelations[chain_id - 1, :] = sm.tsa.acf(
                angle_array_node_n_extended, nlags=len(lags) - 1
            )

            print(f"Autocorrelation of chain {chain_id} computed")

        print(f"\nCompleted!\n")

        return pd.DataFrame(np.transpose(Total_autocorrelations), columns=columns)

    else:
        warnings.warn(
            "I can't compute angle autocorrelations since there are no chains available"
        )

In [93]:
df = Total_autocorrelations = angle_autocorrelation_compute(
    data_folder, 1, 10000, 30, spin, length, sigma, burnin
)

display(df)

Computing autocorrelation of node 1 over 5 chains with lag max 100...

Autocorrelation of chain 1 computed
Autocorrelation of chain 2 computed
Autocorrelation of chain 3 computed
Autocorrelation of chain 4 computed
Autocorrelation of chain 5 computed

Completed!



,chain 1,chain 2,chain 3,chain 4,chain 5
0,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.957157,0.958168,0.956765,0.957718,0.957431
2,0.917625,0.919432,0.916885,0.918424,0.918157
3,0.881058,0.883444,0.880106,0.881859,0.881708
4,0.846809,0.849820,0.845843,0.847761,0.847530
...,...,...,...,...,...
95,0.090953,0.093506,0.103226,0.098626,0.105362
96,0.089114,0.092081,0.101659,0.096437,0.103903
97,0.087303,0.090696,0.099904,0.094396,0.102640
98,0.085400,0.089299,0.098237,0.092399,0.101265


In [95]:
# final average over number of chains
autocorr = df.mean(axis=1)

display(autocorr)

0     1.000000
1     0.957448
2     0.918105
3     0.881635
4     0.847553
        ...   
95    0.098335
96    0.096639
97    0.094988
98    0.093320
99    0.091634
Length: 100, dtype: float64